In [1]:
import sys
import numpy as np
import pandas
from multiprocessing import Pool
sys.path.append("../DependenceFitter-V2")

In [2]:
from drw_toolkit import hampel_filter, simu_fit_mle

In [3]:
df_burke = pandas.read_pickle("df_burke.pkl")
df_ren = pandas.read_pickle("df_ren.pkl")
df_combined = pandas.concat([df_burke,df_ren], axis=0)
df_combined = df_combined.reset_index(drop=True)
df_combined

,name,ra,dec,x,y,yerr,band,z,log_M_BH,log_M_BH_e,log_L_bol,log_L_bol_e,wl_rest,tau_mle,sigma_mle,SNR_mle,Delta_AIC_low,Delta_AIC_hi,ncadence,baseline
0,2.5873.82,79.119930,-68.617328,"[201.66, 202.68, 218.62, 218.81, 230.75, 261.6...","[17.679, 17.639, 17.613, 17.627, 17.652, 17.53...","[0.036, 0.024, 0.034, 0.019, 0.032, 0.017, 0.0...",V,0.460000,8.410000,0.450000,46.156611,0.00,3.574269,2.697895,0.059318,2.583648,1027.271893,3.544879,957,2720.060000
1,NGC 4395,186.453600,33.546860,"[1899.327623455907, 1899.348457374554, 1899.36...","[1590.7554931640625, 1591.3857421875, 1594.143...","[2.868915319442749, 2.806187391281128, 2.73403...",TESS,0.001064,3.959041,0.073923,40.726611,0.03,3.928957,0.337577,3.256124,1.614046,899.991645,5.741561,1202,27.146159
2,25.3469.117,75.444510,-67.544948,"[200.83, 208.77, 218.77, 219.82, 220.77, 231.6...","[18.002, 18.005, 18.066, 18.036, 18.071, 18.07...","[0.028, 0.019, 0.016, 0.016, 0.027, 0.016, 0.0...",V,0.380000,8.150000,0.450000,45.666611,0.00,3.598743,2.626083,0.077122,3.141132,422.193614,4.161477,371,2722.680000
3,25.3712.72,75.723550,-67.429567,"[208.77, 218.77, 219.82, 220.77, 231.69, 233.7...","[18.872, 18.789, 18.804, 18.761, 18.768, 18.77...","[0.049, 0.036, 0.033, 0.056, 0.031, 0.033, 0.0...",V,2.170000,9.040000,0.380000,46.796611,0.00,3.237563,3.135379,0.065994,1.706505,315.776499,2.452306,360,2709.760000
4,30.11301.499,87.423470,-69.737740,"[294.71, 296.72, 305.75, 306.64, 308.69, 315.7...","[19.225, 19.377, 19.412, 19.274, 19.331, 19.03...","[0.055, 0.143, 0.135, 0.048, 0.05, 0.099, 0.25...",V,0.460000,8.140000,0.560000,45.306611,0.00,3.574269,2.254509,0.076868,0.714239,116.494084,6.186005,288,2617.980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828,2MASXJ23075724+4016393,46.988579,40.277531,"[58254.4857523, 58257.4661227, 58266.3835995, ...","[16.6010189, 16.5525761, 16.5675964, 16.529315...","[0.0154059865, 0.0152674653, 0.0153089417, 0.0...",r,0.072658,7.610000,0.100000,45.000000,0.20,3.773610,2.772566,0.196141,5.349337,1566.370352,3.239898,908,2086.673565
829,NGC7603,49.736066,0.243875,"[58277.4200347, 58286.4657523, 58289.4615625, ...","[13.8594236, 13.8984156, 13.9282627, 13.842283...","[0.0143916849, 0.0143387541, 0.0142988423, 0.0...",r,0.028717,8.590000,0.100000,44.820000,0.20,3.791775,2.383821,0.104416,2.421099,247.164264,4.726324,291,2006.708044
830,RHS61,51.476032,21.887190,"[58268.4465856, 58316.4561921, 58319.4464931, ...","[15.9795561, 15.9858809, 15.9876451, 16.015884...","[0.0130429454, 0.0130603323, 0.013065204, 0.01...",r,0.120184,8.390000,0.100000,45.660000,0.20,3.754782,2.544340,0.128692,5.545986,1580.572508,3.817586,944,2065.714352
831,PG2349-014,57.983841,-1.153689,"[58282.4785532, 58288.4621759, 58298.4417593, ...","[16.4072914, 16.4150276, 16.4393673, 16.452188...","[0.0146189751, 0.0146500291, 0.0147498203, 0.0...",r,0.174002,8.700000,0.100000,45.990000,0.20,3.734402,2.445592,0.091428,2.604482,383.838715,4.535227,378,2033.639039


In [4]:
repeat_times = 500
noise = 0.25

In [5]:
df_mock = pandas.DataFrame(data=[])
df_mock["tau_out"] = pandas.Series([None] * 12, dtype="object")
df_mock["sigma_out"] = pandas.Series([None] * 12, dtype="object")
df_mock["Delta_AIC_low"] = pandas.Series([None] * 12, dtype="object")
df_mock["Delta_AIC_hi"] = pandas.Series([None] * 12, dtype="object")
df_mock["SNR"] = pandas.Series([None] * 12, dtype="object")


In [6]:
%%time
num = 0
for df_name in ["burke", "ren", "combined"]:
    df = eval(f"df_{df_name}")
    z = np.repeat(df.z.to_numpy(),repeat_times)
    t = np.repeat(df.x.to_numpy(),repeat_times)
    yerr = np.repeat(df.yerr.to_numpy(),repeat_times)
    sigma = np.repeat(df.sigma_mle.to_numpy(),repeat_times)
    for dependence_code in range(3):
        if dependence_code == 0:
            x = np.repeat(df.log_M_BH.to_numpy(),repeat_times)
            xerr = np.repeat(df.log_M_BH_e.to_numpy(),repeat_times)             
            tau_in = np.random.normal(0.5*np.random.normal(x, xerr) - 1.7, noise) + np.log10(1+z)
        elif dependence_code == 1:
            x = np.repeat(df.log_M_BH.to_numpy(),repeat_times)
            xerr = np.repeat(df.log_M_BH_e.to_numpy(),repeat_times)             
            tau_in = np.random.normal(0.38*np.random.normal(x, xerr) - 1.01, noise) + np.log10(1+z)    
        elif dependence_code == 2:
            x = np.repeat(df.log_L_bol.to_numpy(),repeat_times)
            xerr = np.repeat(df.log_L_bol_e.to_numpy(),repeat_times)             
            y = np.repeat(df.wl_rest.to_numpy(),repeat_times)
            tau_in = np.random.normal(0.72*np.random.normal(x, xerr) + 1.19*y - 34.2, noise) + np.log10(1+z)
        pool = Pool(8)
        args = zip(t, tau_in, sigma, yerr)
        log_pars = pool.starmap(simu_fit_mle, args)
        pool.close()
        pool.join()
        pars0 = np.array(log_pars)
        tau_out, sigma_out, Delta_AIC_low, Delta_AIC_hi, SNR= pars0.T
        df_mock.at[num, "df_name"] = df_name
        df_mock.at[num, "dependence_code"] = dependence_code
        df_mock.at[num, "tau_out"] = np.reshape(tau_out, (-1, repeat_times))
        df_mock.at[num, "sigma_out"] = np.reshape(sigma_out, (-1, repeat_times))
        df_mock.at[num, "Delta_AIC_low"] = np.reshape(Delta_AIC_low, (-1, repeat_times))
        df_mock.at[num, "Delta_AIC_hi"] = np.reshape(Delta_AIC_hi, (-1, repeat_times))
        df_mock.at[num, "SNR"] = np.reshape(SNR, (-1, repeat_times))
        num+=1

CPU times: user 18.7 s, sys: 17min 58s, total: 18min 16s
Wall time: 8h 15s


In [7]:
df_mock.to_pickle("df_mock_25noise.pkl")

,tau_out,sigma_out,Delta_AIC_low,Delta_AIC_hi,SNR,df_name,dependence_code
0,"[[2.7781494238651545, 1.978370254392399], [0.2...","[[0.05572030358898314, 0.03525273331426092], [...","[[1402.1933494692694, 841.3719396821377], [970...","[[304.5354797607665, 605.7806896791292], [970....","[[3.332899589439246, 2.1346266595525787], [1.6...",burke,0.0
1,"[[2.0122125040513406, 2.2595897554148605], [0....","[[0.0293671243867593, 0.047858240815882566], [...","[[751.6903598813628, 1096.395950390936], [361....","[[418.11775495361417, 712.6611211929021], [362...","[[1.786250643276596, 2.9096478314625656], [0.9...",burke,1.0
2,"[[2.343763162981929, 3.3386088732088997], [0.2...","[[0.01274183653429488, 0.04494231170372856], [...","[[355.2873407300913, 987.4606297760904], [1058...","[[44.37702758613341, 56.42937899748449], [1058...","[[0.7852691120934412, 2.753161723785893], [2.1...",burke,2.0
3,"[[1.8990505430462454, 1.407914619177312], [1.6...","[[0.15785301755893824, 0.1492772910388014], [0...","[[651.4281085401699, 467.44766233567157], [637...","[[647.9272747187955, 485.32585860714386], [619...","[[11.712858098985842, 9.4791628742994], [10.86...",ren,0.0
4,"[[1.3363062098262373, 1.4933698818946002], [1....","[[0.12072369749776922, 0.12929354792985467], [...","[[414.61390511728285, 457.6802606998328], [703...","[[429.1921340367747, 474.0178025217206], [655....","[[8.96271895093357, 9.064243815890926], [10.04...",ren,1.0
5,"[[1.3690321994493444, 1.832244160752639], [1.8...","[[0.13254739573432053, 0.1468519837952586], [0...","[[408.2106197055723, 630.3056896858775], [680....","[[426.9648344792846, 604.5671756634932], [512....","[[9.84341663480249, 10.893862535456009], [8.92...",ren,2.0
6,"[[2.5551337787390525, 2.7920542945131204], [2....","[[0.09469383803869072, 0.11135884550553328], [...","[[224.50784790159238, 286.02700246831535], [29...","[[181.1888123687592, 235.28410932556997], [276...","[[4.979558999511766, 5.892867156820657], [5.79...",stone,0.0
7,"[[2.8993640108314405, 2.495797320528969], [2.0...","[[0.14977486439677537, 0.1059740719183005], [0...","[[382.85756835138005, 257.9294037610641], [194...","[[283.48869295831526, 229.01954750710885], [18...","[[7.468853489948915, 5.305286269197941], [3.17...",stone,1.0
8,"[[3.0898284551816446, 3.423400730372586], [2.5...","[[0.11315254148983901, 0.0865389950426966], [0...","[[427.4024401683133, 311.89596270991154], [235...","[[279.9551325782777, 119.05246022801037], [186...","[[6.009380191858578, 4.8912728773834795], [3.4...",stone,2.0
9,"[[2.736523506935627, 2.143834782158304], [-0.0...","[[0.07865269233122474, 0.05312525642552311], [...","[[1553.1375898003048, 1164.9641611415939], [55...","[[723.1272209073304, 659.0816238890086], [558....","[[4.775019366290692, 3.2149057986104927], [1.1...",combined,0.0
